In [ ]:
import json
import pickle
from itertools import chain

import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
%matplotlib inline

from neuron import h
from dlutils import utils
from dlutils.cell import Cell, branch_order
from dlutils.synapse import AMPANMDAExp2Synapse
from dlutils.spine import Spine

h.load_file('stdrun.hoc');

### General parameters

In [ ]:
optimization_folder = '/Users/daniele/Postdoc/Research/Janelia/01_model_optimization/'

# config_file = 'synaptic_cooperativity_thorny_passive.json'
# config_file = 'synaptic_cooperativity_thorny_active_with_TTX.json'
# config_file = 'synaptic_cooperativity_thorny_active.json'
# config_file = 'synaptic_cooperativity_a-thorny_passive.json'
# config_file = 'synaptic_cooperativity_a-thorny_active_with_TTX.json'
config_file = 'synaptic_cooperativity_a-thorny_active.json'
config = json.load(open(config_file, 'r'))

cell_type = config['cell_type']
prefix = cell_type[0].upper() + cell_type[1:]
base_folder = optimization_folder + prefix + '/' + config['cell_name'] + '/' + config['optimization_run'] + '/'
swc_file = config['swc_file']
cell_name = config['cell_name'] + '_'
individual = config['individual']

swc_file = base_folder + swc_file
params_file = base_folder + 'individual_{}.json'.format(individual)
config_file = base_folder + 'parameters.json'

passive = config['passive']
with_TTX = config['with_TTX']
replace_axon = True
add_axon_if_missing = True
parameters = json.load(open(params_file, 'r'))
mechanisms = utils.extract_mechanisms(config_file, cell_name)
sim_pars = pickle.load(open(base_folder + 'simulation_parameters.pkl','rb'))
replace_axon = sim_pars['replace_axon']
add_axon_if_missing = not sim_pars['no_add_axon']

### Functions used to describe the removal of the Mg block from the NMDA synapse

#### Maex & De Schutter
Maex, R., & De Schutter, E. (1998). Synchronization of Golgi and granule cell firing in a detailed network model of the cerebellar granule cell layer. Journal of Neurophysiology, 80(5), 2521–2537. http://doi.org/10.1152/jn.1998.80.5.2521

#### Jahr & Stevens
Jahr, C. E., & Stevens, C. F. (1990). A quantitative description of NMDA receptor-channel kinetic behavior. The Journal of Neuroscience, 10(6), 1830–1837.

Jahr, C. E., & Stevens, C. F. (1990). Voltage dependence of NMDA-activated macroscopic conductances predicted by single-channel kinetics. The Journal of Neuroscience, 10(9), 3178–3182. http://doi.org/10.1523/JNEUROSCI.10-09-03178.1990

#### Harnett
Harnett, M. T., Makara, J. K., Spruston, N., Kath, W. L., & Magee, J. C. (2012). Synaptic amplification by dendritic spines enhances input cooperativity. Nature, 491(7425), 599–602. http://doi.org/10.1038/nature11554

There is a mistake in the definition of the Mg removal function: the units of the expression found in the paper are incorrect. The correct function is the one found here and is actually also found in the second of Jahr & Stevens's papers (equation 5).

In [ ]:
v = np.arange(-100, 50)
extMgConc = config['NMDA']['extMgConc']

mg_maex_orig = lambda v: 1. / (1. + 0.2801 * extMgConc * np.exp(-0.062 * (v - 10)))

mg_jahr_stevens_orig = lambda v: 1. / (1 + (extMgConc / 9.888) * np.exp(0.09137 * (2.222 - v)))

mg_harnett = lambda v: 1. / (1. + (extMgConc / 3.57) * np.exp(-0.062 * v))

plt.figure(figsize=(8,4))
plt.plot(v, mg_harnett(v), 'k--', lw=3, label='Harnett')
plt.plot(v, mg_maex_orig(v), color=[0,.5,0], label='Original Maex & De Schutter')

if config['NMDA']['model'] == 'MDS':
    alpha_vspom = config['NMDA']['alpha_vspom'] # -0.124
    v0_block    = config['NMDA']['v0_block']    # -10
    eta         = config['NMDA']['eta']         # 0.02801
    mg_maex_mod = lambda v: 1. / (1. + eta * extMgConc * np.exp(alpha_vspom * (v - v0_block)))
    plt.plot(v, mg_maex_mod(v), color=[.5,1,.5], label='Modified Maex & De Schutter')

plt.plot(v, mg_jahr_stevens_orig(v), color=[.65,0,0], label='Original Jahr & Stevens')

if config['NMDA']['model'] == 'JS':
    Kd    = config['NMDA']['Kd']    # (mM)
    gamma = config['NMDA']['gamma'] # (mV^-1)
    sh    = config['NMDA']['sh']    # (mV)
    mg_jahr_stevens_mod = lambda v: 1. / (1 + (extMgConc / Kd) * np.exp(gamma * (sh - v)))
    plt.plot(v, mg_jahr_stevens_mod(v), color=[1,.5,.5], label='Modified Jahr & Stevens')

plt.xlabel('Voltage (mV)')
plt.ylabel('Mg unblock')
plt.legend(loc='best');

### Instantiate the cell

In [ ]:
cell = Cell('CA3_cell_%d' % int(np.random.uniform()*1e5), swc_file, parameters, mechanisms)
cell.instantiate(replace_axon, add_axon_if_missing, force_passive=passive, TTX=with_TTX)
section_num = config['section_num']
parent_section = cell.morpho.apic[section_num].parentseg().sec
# this way the sections are sorted
sections = parent_section.children()[2::-1]
n_apical_branches = len(sections)
dend_ids = [int(sec.name().split('[')[-1][:-1]) for sec in sections]
Ra = sections[0].Ra * config['Ra_neck_coeff']
print('Branch order of parent section {}: {}.'.format(parent_section.name(), branch_order(parent_section)))
for section in sections:
    print('Branch order of section {}: {}.'.format(section.name(), branch_order(section)))
    print('Length of section {}: {:.0f} um.'.format(section.name(), section.L))

### Instantiate the spines

In [ ]:
# in the Harnett paper, the head is spherical with a diameter of 0.5 um: a cylinder
# with diameter and length equal to 0.5 has the same (outer) surface area as the sphere
head_L = config['spine']['head_L']           # [um]
head_diam = config['spine']['head_diam']     # [um]
neck_L = config['spine']['neck_L']           # [um]
neck_diam = config['spine']['neck_diam']     # [um]
spine_distance = config['spine_distance']    # [um] distance between neighboring spines
n_spines = config['n_spines']                # number of spines
L = spine_distance * (n_spines - 1)
norm_L = L / section.L
spines = []
if section_num == 0:
    for section in sections:
        spines.append([Spine(section, x, head_L, head_diam, neck_L, neck_diam, Ra, i) \
                  for i,x in enumerate(np.linspace(0.5 - norm_L/2, 0.5 + norm_L/2, n_spines))])
else:
    for section in sections:
        spines.append([Spine(section, x, head_L, head_diam, neck_L, neck_diam, Ra, i) \
                  for i,x in enumerate(np.linspace(0.2 - norm_L/2, 0.2 + norm_L/2, n_spines))])

for group in spines:
    for spine in group:
        spine.instantiate()

#### Check the location of the spines in terms of distinct segments

#### Show where the spines are located on the dendritic tree

In [ ]:
plt.figure(figsize=(10,10))
for sec in chain(cell.morpho.apic, cell.morpho.basal):
    if sec in cell.morpho.apic:
        color = 'k'
    else:
        color = 'b'
    lbl = sec.name().split('.')[1].split('[')[1][:-1]
    n = sec.n3d()
    sec_coords = np.zeros((n,2))
    for i in range(n):
        sec_coords[i,:] = np.array([sec.x3d(i), sec.y3d(i)])
    middle = int(n / 2)
    plt.text(sec_coords[middle,0], sec_coords[middle,1], lbl, \
             fontsize=14, color='m')
    plt.plot(sec_coords[:,0], sec_coords[:,1], color, lw=1)
for group in spines:
    for spine in group:
        plt.plot(spine._points[:,0], spine._points[:,1], 'r.')
plt.axis('equal');

### Insert a synapse into each spine

In [ ]:
MG_MODELS = {'MDS': 1, 'HRN': 2, 'JS': 3}
Mg_unblock_model = config['NMDA']['model']

E = 0        # [mV]

AMPA_taus = config['AMPA']['time_constants']
NMDA_taus = config['NMDA']['time_constants']
weights = np.array([config['AMPA']['weight'], config['NMDA']['weight']])

print('AMPA:')
print('    tau_rise = {:.3f} ms'.format(AMPA_taus['tau1']))
print('   tau_decay = {:.3f} ms'.format(AMPA_taus['tau2']))
print('NMDA:')
print('    tau_rise = {:.3f} ms'.format(NMDA_taus['tau1']))
print('   tau_decay = {:.3f} ms'.format(NMDA_taus['tau2']))

synapses = [[AMPANMDAExp2Synapse(spine.head, 1, E, weights, AMPA = AMPA_taus, \
                                NMDA = NMDA_taus) for spine in group] for group in spines]

for group in synapses:
    for syn in group:
        syn.nmda_syn.mg_unblock_model = MG_MODELS[Mg_unblock_model]
        if Mg_unblock_model == 'MDS':
            syn.nmda_syn.alpha_vspom = config['NMDA']['alpha_vspom']
            syn.nmda_syn.v0_block = config['NMDA']['v0_block']
            syn.nmda_syn.eta = config['NMDA']['eta']
        elif Mg_unblock_model == 'JS':
            syn.nmda_syn.Kd = config['NMDA']['Kd']
            syn.nmda_syn.gamma = config['NMDA']['gamma']
            syn.nmda_syn.sh = config['NMDA']['sh']

if Mg_unblock_model == 'MDS':
    print('\nUsing Maex & De Schutter Mg unblock model. Modified parameters:')
    print('       alpha = {:.3f} 1/mV'.format(synapses[0][0].nmda_syn.alpha_vspom))
    print('    v0_block = {:.3f} mV'.format(synapses[0][0].nmda_syn.v0_block))
    print('         eta = {:.3f}'.format(synapses[0][0].nmda_syn.eta))
elif Mg_unblock_model == 'JS':
    print('\nUsing Jahr & Stevens Mg unblock model. Modified parameters:')
    print('          Kd = {:.3f} 1/mV'.format(synapses[0][0].nmda_syn.Kd))
    print('       gamma = {:.3f} 1/mV'.format(synapses[0][0].nmda_syn.gamma))
    print('          sh = {:.3f} mV'.format(synapses[0][0].nmda_syn.sh))
elif Mg_unblock_model == 'HRN':
    print('\nUsing Harnett Mg unblock model with default parameters.')

#### Make the recorders

In [ ]:
rec = {}
for lbl in 't','Vsoma','Vdend-parent','Vapic':
    rec[lbl] = h.Vector()
rec['t'].record(h._ref_t)
rec['Vsoma'].record(cell.morpho.soma[0](0.5)._ref_v)
rec['Vdend-parent'].record(parent_section(1)._ref_v)
rec['Vapic'].record(cell.morpho.apic[0](1)._ref_v)
for i,group in enumerate(spines):
    for j,spine in enumerate(group):
        rec['Vdend-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['Vdend-{}-{}-bp'.format(dend_ids[i], j)] = h.Vector()
        rec['Vspine-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['IAMPA-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['gAMPA-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['INMDA-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['gNMDA-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['MgBlock-{}-{}'.format(dend_ids[i], j)] = h.Vector()
        rec['Vdend-{}-{}'.format(dend_ids[i], j)].record(spine._sec(spine._sec_x)._ref_v)
        rec['Vdend-{}-{}-bp'.format(dend_ids[i], j)].record(spine._sec(0)._ref_v)
        rec['Vspine-{}-{}'.format(dend_ids[i], j)].record(spine.head(0.5)._ref_v)
        rec['IAMPA-{}-{}'.format(dend_ids[i], j)].record(synapses[i][j].syn[0]._ref_i)
        rec['gAMPA-{}-{}'.format(dend_ids[i], j)].record(synapses[i][j].syn[0]._ref_g)
        rec['INMDA-{}-{}'.format(dend_ids[i], j)].record(synapses[i][j].syn[1]._ref_i)
        rec['gNMDA-{}-{}'.format(dend_ids[i], j)].record(synapses[i][j].syn[1]._ref_g)
        rec['MgBlock-{}-{}'.format(dend_ids[i], j)].record(synapses[i][j].syn[1]._ref_mgBlock)

### Compute somatic and dendritic input resistance

In [ ]:
stim = {'soma': h.IClamp(cell.morpho.soma[0](0.5))}
for dend_id,section,spine_group in zip(dend_ids, sections, spines):
    stim['dend-{}'.format(dend_id)] = h.IClamp(section(spine_group[0]._sec_x))
for i,k in enumerate(stim):
    stim[k].delay = 500 + i*1000
    stim[k].amp = -0.1
    stim[k].dur = 300
    h.tstop = stim[k].delay
h.tstop += 1000
print('Simulation duration: {} ms.'.format(h.tstop))
h.run()
t = np.array(rec['t'])
V = {'soma': np.array(rec['Vsoma'])}
for dend_id in dend_ids:
    V['dend-{}'.format(dend_id)] = np.array(rec['Vdend-{}-0'.format(dend_id)])
R = {}
for k in V:
    idx, = np.where((t > stim[k].delay) & (t < stim[k].delay + stim[k].dur))
    V0 = V[k][idx[0]-10]
    V1 = V[k][idx[-1]]
    R[k] = (V1 - V0) / stim[k].amp
    stim[k].amp = 0
    stim[k].dur = 0
    stim[k].delay = 1e6
    
print('   Somatic RMP: {:.1f} mV.'.format(V['soma'][-1]))
for i,dend_id in enumerate(dend_ids):
    print('Apical #{} RMP: {:.1f} mV.'.format(dend_id, V['dend-{}'.format(dend_id)][-1]))

col = ['m', 'g']
print('\n   Somatic Rin: {:.0f} MOhm.'.format(R['soma']))
for i,dend_id in enumerate(dend_ids):
    print('Apical #{} Rin: {:.0f} MOhm.'.format(dend_id, R['dend-{}'.format(dend_id)]))

plt.plot(t, V['soma'], 'k', lw=1, label='Soma')
for i,dend_id in enumerate(dend_ids):
    plt.plot(t, V['dend-{}'.format(dend_id)], col[i], lw=1, label='Dendrite {}'.format(dend_id))
plt.xlabel('Time (ms)')
plt.ylabel('Vm (mV)')
plt.legend(loc='best');

#### Compute the presynaptic spike times

In [ ]:
t0 = 1000.
dt = 1000.
with_somatic_current_injection = True
if with_somatic_current_injection:
    spike_times = [np.sort(t0 + n_spines * dt - np.arange(i) * dt) for i in range(n_spines+1, 0, -1)]
    spike_times = spike_times[:-1]
else:
    spike_times = [np.sort(t0 + (n_spines - 1) * dt - np.arange(i) * dt) for i in range(n_spines, 0, -1)]
poisson = False
sequential = True
if sequential:
    for i in range(n_spines):
        for j in range(n_spines-i):
            spike_times[i][j] += i * 0.3
elif poisson:
    # frequency of the incoming spikes
    F = 1 / 0.3e-3
    for i in range(n_spines):
        ISI = - np.log(np.random.uniform(size=n_spines*2)) / F
        spks = np.cumsum(ISI)
        for j in range(n_spines - i):
            spike_times[j][i] += spks[j]

for group in synapses:
    for syn,spks in zip(group, spike_times):
        syn.set_presynaptic_spike_times(spks)
        
if with_somatic_current_injection:
    stim['soma'].amp = 0.2
    stim['soma'].dur = 100
    stim['soma'].delay = spike_times[0][-1]

In [ ]:
spike_times

#### Run the simulation

In [ ]:
h.cvode_active(1)
h.tstop = t0 + n_spines * dt
if with_somatic_current_injection:
    h.tstop += dt
h.run()

#### Get the data from the recorders

In [ ]:
t = np.array(rec['t'])
iampa = np.array([[np.array(rec['IAMPA-{}-{}'.format(dend_id, i)])*1e3 for i in range(n_spines)] for dend_id in dend_ids])
inmda = np.array([[np.array(rec['INMDA-{}-{}'.format(dend_id, i)])*1e3 for i in range(n_spines)] for dend_id in dend_ids])
gampa = np.array([[np.array(rec['gAMPA-{}-{}'.format(dend_id, i)])*1e3 for i in range(n_spines)] for dend_id in dend_ids])
gnmda = np.array([[np.array(rec['gNMDA-{}-{}'.format(dend_id, i)])*1e3 for i in range(n_spines)] for dend_id in dend_ids])
MgBlock = np.array([[np.array(rec['MgBlock-{}-{}'.format(dend_id, i)]) for i in range(n_spines)] for dend_id in dend_ids])
Vspine = np.array([[np.array(rec['Vspine-{}-{}'.format(dend_id, i)]) for i in range(n_spines)] for dend_id in dend_ids])
Vdend = np.array([[np.array(rec['Vdend-{}-{}'.format(dend_id, i)]) for i in range(n_spines)] for dend_id in dend_ids])
Vdend_bp = np.array([[np.array(rec['Vdend-{}-{}-bp'.format(dend_id, i)]) for i in range(n_spines)] for dend_id in dend_ids])
Vdend_parent = np.array(rec['Vdend-parent'])
Vapic = np.array(rec['Vapic'])
Vsoma = np.array(rec['Vsoma'])

In [ ]:
if with_somatic_current_injection:
    fig,ax = plt.subplots(1, 1, figsize=(10,6))
    t0 = stim['soma'].delay
    idx, = np.where((t > spike_times[0][-2] - 20) & (t < spike_times[0][-2] + 150))
    ax.plot(t[idx] - spike_times[0][-2], Vsoma[idx], 'k', lw=1, label='Without somatic stimulus')
    idx, = np.where((t > spike_times[0][-1] - 20) & (t < spike_times[0][-1] + 150))
    ax.plot(t[idx] - spike_times[0][-1], Vsoma[idx], 'r', lw=1, label='With somatic stimulus')
    yl = ax.get_ylim()
    dy = yl[1] - yl[0]
    ax.plot([0, stim['soma'].dur], yl[0] + np.zeros(2), color=[.8,.8,1], lw=5)
    ax.text(stim['soma'].dur/2, yl[0] - dy/30, 'SOMATIC STIMULUS', horizontalalignment='center', verticalalignment='top')
    plt.xlabel('Time since presynaptic spike arrival (ms)')
    plt.ylabel('Membrane voltage (mV)')
    ax.set_ylim([yl[0] - dy/10, yl[1]])
    ax.legend(loc='best');

#### Measure the amplitude ratio when the input is synaptic

In [ ]:
idx, = np.where((t > 990) & (t < 1200))
EPSP_spine = np.array([np.max(Vspine[i,0,idx]) - Vspine[i,0,-1] for i in range(n_apical_branches)])
EPSP_dend = np.array([np.max(Vdend[i,0,idx]) - Vdend[i,0,-1] for i in range(n_apical_branches)])
AR = EPSP_spine / EPSP_dend
for i,dend_id in enumerate(dend_ids):
    R['neck-{}'.format(dend_id)] = (AR[i] - 1) * R['dend-{}'.format(dend_id)]
    print('=== Apical branch #{} ==='.format(dend_id))
    print('            AR: {:.3f}'.format(AR[i]))
    print(' Dendritic Rin: {:.0f} MOhm.'.format(R['dend-{}'.format(dend_id)]))
    print('Spine neck Rin: {:.0f} MOhm.'.format(R['neck-{}'.format(dend_id)]))
    print('')

fig,ax = plt.subplots(1, n_apical_branches, sharex=True, sharey=True, figsize=(10,4))
for i,dend_id in enumerate(dend_ids):
    ax[i].plot(t[idx], Vspine[i, 0,idx], 'k', label='Spine')
    ax[i].plot(t[idx], Vdend[i, 0,idx], 'r', label='Dendrite')
    ax[i].set_xlabel('Time (ms)')
    ax[i].set_title('Apical branch #{}'.format(dend_id))
ax[0].set_ylabel('Vm (mV)')
ax[0].legend(loc='best');

#### Measure the dendritic EPSPs and the NMDA conductance values

In [ ]:
V_soma_pks = []
V_dend_pks = [[] for _ in range(n_apical_branches)]
G_AMPA_pks = [[] for _ in range(n_apical_branches)]
G_NMDA_pks = [[] for _ in range(n_apical_branches)]
for spk in spike_times[0][:n_spines]:
    idx, = np.where((t > spk) & (t < spk + 200))
    V_soma_pks.append(np.argmax(Vsoma[idx]) + idx[0])
    for i in range(n_apical_branches):
        V_dend_pks[i].append(np.argmax(Vdend[i, 0, idx]) + idx[0])
        G_AMPA_pks[i].append(np.argmax(gampa[i, 0, idx]) + idx[0])
        G_NMDA_pks[i].append(np.argmax(gnmda[i, 0, idx] * MgBlock[i, 0,idx]) + idx[0])
V_soma_pks = np.array(V_soma_pks)
V_dend_pks = np.array(V_dend_pks)
G_AMPA_pks = np.array(G_AMPA_pks)
G_NMDA_pks = np.array(G_NMDA_pks)
dV_soma = Vsoma[V_soma_pks] - Vsoma[-1]
dV_dend = np.array([Vdend[i, 0, V_dend_pks[i]] - Vdend[i, 0, -1] for i in range(n_apical_branches)])
dG_AMPA = np.array([gampa[i, 0, G_AMPA_pks[i]] for i in range(n_apical_branches)])
dG_NMDA = np.array([gnmda[i, 0, G_NMDA_pks[i]] * MgBlock[i, 0, G_NMDA_pks[i]] for i in range(n_apical_branches)])

#### Plot the time course of the dendritic EPSPs and the NMDA conductance for increasing number of synaptic inputs

In [ ]:
cmap = plt.get_cmap('viridis', n_spines)
window = [10, 100]
fig,ax = plt.subplots(n_apical_branches, 3, figsize=(17,5*n_apical_branches), sharex=True)

for branch_idx in range(n_apical_branches):
    for i,spk in enumerate(spike_times[0][:n_spines]):
        idx = (t > spk - window[0]) & (t < spk + window[1])
        ax[branch_idx,0].plot(t[idx] - spk, Vsoma[idx], color=cmap(i))
        ax[branch_idx,1].plot(t[idx] - spk, Vdend[branch_idx, 0, idx], color=cmap(i))
        ax[branch_idx,2].plot(t[idx] - spk, gnmda[branch_idx, 0, idx] * MgBlock[branch_idx, 0, idx], color=cmap(i))
    for i,spk in enumerate(spike_times[0][:n_spines]):
        ax[branch_idx,0].plot(t[V_soma_pks[i]] - spk, Vsoma[V_soma_pks[i]], 'ro', markerfacecolor='w', linewidth=2)
        ax[branch_idx,1].plot(t[V_dend_pks[branch_idx, i]] - spk, Vdend[branch_idx, 0, V_dend_pks[branch_idx, i]], \
                 'ro', markerfacecolor='w', linewidth=2)
        ax[branch_idx,2].plot(t[G_NMDA_pks[branch_idx, i]] - spk, gnmda[branch_idx, 0, G_NMDA_pks[branch_idx, i]] * \
                 MgBlock[branch_idx, 0, G_NMDA_pks[branch_idx, i]], 'ro', markerfacecolor='w', linewidth=2)
    ax[branch_idx,0].set_ylabel('Somatic voltage (mV)')
    ax[branch_idx,1].set_ylabel('Dendritic voltage (mV)')
    ax[branch_idx,2].set_ylabel('NMDA conductance (nS)')
    ax[branch_idx,1].set_title('Apical branch #{}'.format(dend_ids[branch_idx]))

for i in range(3):
    ax[n_apical_branches-1,i].set_xlabel('Time (ms)');

#### Plot the time course of the EPSP at each spine for increasing number of synaptic inputs

In [ ]:
cmap = plt.get_cmap('viridis', n_spines)
window = [10, 100]

for branch_idx in range(n_apical_branches):
    fig,ax = plt.subplots(3, 3, figsize=(14,10), sharex=True)
    for n,spk in enumerate(spike_times[0][:n_spines]):
        idx = (t > spk - window[0]) & (t < spk + window[1])
        for i in range(3):
            for j in range(3):
                k = i*3 + j
                lbl = '{} input'.format(n+1) if n == 0 else '{} inputs'.format(n+1)
                ax[i,j].plot(t[idx] - spk, Vspine[branch_idx, k, idx], color=cmap(n), label=lbl)
                ax[i,j].set_title('Spine #{}'.format(k+1))
            ax[i,0].set_ylabel('Spine head voltage (mV)')
            ax[2,i].set_xlabel('Time (ms)')
    ax[0,0].legend(loc='best');

#### Plot the time course of the NMDA conductance in each spine for increasing number of synaptic inputs

In [ ]:
cmap = plt.get_cmap('rainbow', n_spines)
window = [10, 100]

for branch_idx in range(n_apical_branches):
    fig,ax = plt.subplots(3, 3, figsize=(14,10), sharex=True)
    for n,spk in enumerate(spike_times[0][:n_spines]):
        idx = (t > spk - window[0]) & (t < spk + window[1])
        for i in range(3):
            for j in range(3):
                k = i*3 + j
                lbl = '{} input'.format(n+1) if n == 0 else '{} inputs'.format(n+1)
                ax[i,j].plot(t[idx] - spk, gnmda[branch_idx, k, idx] * MgBlock[branch_idx, k, idx], \
                             color=cmap(n), label=lbl)
                ax[i,j].set_title('Spine #{}'.format(k+1))
            ax[i,0].set_ylabel('NMDA conductance (nS)')
            ax[2,i].set_xlabel('Time (ms)')
    ax[0,0].legend(loc='best');

#### Plot the amplitude of the dendritic EPSPs and of the NMDA conductance as a function of the number of synaptic inputs

In [ ]:
n = 1 + np.arange(n_spines)
fig,ax = plt.subplots(n_apical_branches, 3, figsize=(17,5*n_apical_branches), sharex=True)
for branch_idx in range(n_apical_branches):
    ax[branch_idx, 0].plot(n, n * dV_dend[branch_idx, 0], 'rv-', lw=1, markerfacecolor='w', \
                           markersize=8, markeredgewidth=1.5, label='Linear prediction')
    ax[branch_idx, 0].plot(n, dV_dend[branch_idx], 'ko-', lw=1, markerfacecolor='w', \
                           markersize=7, markeredgewidth=1.5, label='Measured')
    ax[branch_idx, 1].plot(n, n * dV_soma[0], 'rv-', lw=1, markerfacecolor='w', markersize=8, \
                           markeredgewidth=1.5, label='Linear prediction')
    ax[branch_idx, 1].plot(n, dV_soma, 'ko-', lw=1, markerfacecolor='w', markersize=7, \
                           markeredgewidth=1.5, label='Measured')
    ax[branch_idx, 2].plot(n, dG_AMPA[branch_idx], 'bs-', lw=1, markerfacecolor='w', \
                           markersize=7, markeredgewidth=1.5, label='AMPA')
    ax[branch_idx, 2].plot(n, dG_NMDA[branch_idx], 'ko-', lw=1, markerfacecolor='w', \
                           markersize=7, markeredgewidth=1.5, label='NMDA')
    ax[branch_idx, 0].set_ylabel('Dendritic EPSP (mV)')
    ax[branch_idx, 1].set_ylabel('Somatic EPSP (mV)')
    ax[branch_idx, 2].set_ylabel('Conductance (nS)')
for i in range(3):
    ax[1,i].set_xlabel('Input number')
ax[0,0].legend(loc='best')
ax[0,2].legend(loc='best')
for i in range(n_apical_branches):
    ax[i,0].set_ylim([0, np.max([np.max(dV_dend[:,-1]), np.max(dV_dend[:,0])*n_spines]) * 1.1])
    ax[i,1].set_ylim([0, np.max([dV_soma[-1], dV_soma[0]*n_spines]) * 1.1])
    ax[i,2].set_ylim([np.min(dG_NMDA[:,0]) - 0.1, np.max(dG_NMDA[:,-1]) + 0.1]);

In [ ]:
window = [50,200]
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax2.plot(v, mg_jahr_stevens_orig(v), color=[0,0,0], lw=3, label='Original Jahr & Stevens')
ax2.plot(v, mg_jahr_stevens_mod(v), color=[.75,.75,.75], lw=3, label='Modified Jahr & Stevens')
ax2.legend(loc='upper left')
ax2.tick_params(axis='y', labelcolor='tab:green')
ax2.set_ylabel('Mg unblock')

colormaps = 'gray', 'jet'
for n,spk in enumerate(spike_times[0][:n_spines]):
    idx, = np.where((t > spk - window[0]) & (t < spk + window[1]))
    for branch_idx in range(n_apical_branches):
        cmap = plt.get_cmap(colormaps[branch_idx], n_spines)
        ax1.plot(Vdend[branch_idx, 0, idx], Vsoma[idx], color=cmap(n), lw=1)
ax1.set_xlabel('Dendritic EPSP (mV)')
ax1.set_ylabel('Somatic EPSP (mV)')
ax1.tick_params(axis='y', labelcolor='tab:red')

plt.xlim([-80,20]);

#### Save the data

In [ ]:
data_to_save = {'t': t, 'Vsoma': Vsoma, 'Vdend': Vdend, 'Vspine': Vspine,
                'iampa': iampa, 'inmda': inmda, 'gampa': gampa, 'gnmda': gnmda,
                'MgBlock': MgBlock, 'R': R, 'AR': AR, 'spike_times': spike_times,
                'dV_soma': dV_soma, 'dV_dend': dV_dend, 'dG_AMPA': dG_AMPA, 'dG_NMDA': dG_NMDA,
                'V_dend_pks': V_dend_pks, 'V_soma_pks': V_soma_pks, 'G_NMDA_pks': G_NMDA_pks, 
                'G_AMPA_pks': G_AMPA_pks, 'config': config}
filename = 'synaptic_cooperativity_two_branches' + cell_type + '_' + str(individual)
if passive:
    filename += '_passive'
else:
    filename += '_active'
if with_TTX:
    filename += '_with_TTX'
else:
    filename += '_without_TTX'
pickle.dump(data_to_save, open(filename + '.pkl', 'wb'))

In [ ]:
col = 'mg'
for i,dend_id in enumerate(dend_ids):
    plt.plot(t, Vdend[i, 0], col[i], lw=1, label='Apical branch #{}'.format(dend_id))
#     plt.plot(t, Vdend_bp[i, 0], col[i]+'--', lw=1, label='Apical branch #{} BP'.format(dend_id))
plt.plot(t, Vdend_parent, 'r', lw=1, label='Parent branch')
plt.plot(t, Vapic, color=[0,0,1], lw=1, label='Main apical')
plt.plot(t, Vsoma, 'k', lw=1, label='Soma')
plt.legend(loc='best')
# plt.ylim([-50,-35])
plt.xlim([9000, 9050]);